In [1]:
from __future__ import print_function
import argparse
from math import log10
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from model import Net
from model2 import DeepULMNet
from attention import RCAN
# from dataset import get_training_set
import torch.optim.lr_scheduler as lr_scheduler
import torch.nn.functional as F
import torchvision.transforms as transforms
import numpy as np
import time
import os
import matplotlib.pyplot as plt
from torchsummary import summary
from dataset_x8 import get_training_set

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # Have bigger cells in Jupyter Notebook 
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True

# Hyper-parameters

In [3]:
# Settings and hyperparameters
upscale_factor = 4
batchSize = 16
testBatchSize = 500
nEpochs = 200
lr = 0.001
threads = 0
seed = 123

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed(seed)
else:
    device = torch.device("cpu")

# Normalization

### Test to normalize the data (not successful)

In [5]:
# # 0.1171 - 0.2253 07_noisy / 

# transform = transforms.Compose([
#     transforms.Normalize(0.1171, 0.2253)
# ])
# target_transform = transforms.Compose([
#     transforms.Normalize(0.1171, 0.2253)
# ])

In [6]:
# root = 'training_set_numpy/training_set_numpy_07noisy/training_set_32x32_batch_64/'
# root = 'training_set_numpy/training_set_numpy_noisy/training_set_32x32_batch_64/'
# root = 'training_set_numpy/training_set_numpy_no_noise/training_set_32x32_batch_256/'

# train_set = get_training_set(transform=transform, target_transform=None, root = root)
# training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=len(train_set), shuffle=False)

In [7]:
# data = next(iter(training_data_loader))
# print(data[0].mean(), data[0].std())
# data = next(iter(testing_data_loader))
# print(data[0].mean(), data[0].std())

# Data training selection

In [8]:
# root = '../training_set_numpy/training_set_numpy_07noisy/training_set_32x32_batch_64/'
# root = '../training_set_numpy/training_set_numpy_noisy/training_set_32x32_batch_64/'
# root = '../training_set_numpy/training_set_numpy_no_noise/training_set_32x32_batch_256/'
root = '../training_set_np/'

# train_set = get_training_set(transform=transform, target_transform=target_transform, root = root)
train_set = get_training_set(transform=None, target_transform=None, root = root)

# 10000 - 7500 for 07 noise
indices = np.arange(8400)
np.random.shuffle(indices)
training_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=batchSize, shuffle=False, sampler = SubsetRandomSampler(indices[:6300]))
testing_data_loader = DataLoader(dataset=train_set, num_workers=threads, batch_size=testBatchSize, shuffle=False, sampler = SubsetRandomSampler(indices[6300:]))

# Building the model

In [9]:
print('===> Building model')
# model = Net(upscale_factor=upscale_factor)
# model = DeepULMNet()
model = RCAN(n_resgroups = 10, n_resblocks = 20, n_feats = 64, reduction = 16, scale = 8, n_colors = 1)
model = model.cuda()
summary(model, input_size = (1,32,32))
print(model)
criterion = nn.MSELoss().cuda()
criterion2 = nn.L1Loss().cuda()

optimizer = optim.Adam(model.parameters(), lr=lr)
# scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, nEpochs)

===> Building model
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]             640
            Conv2d-2           [-1, 64, 32, 32]          36,928
       BatchNorm2d-3           [-1, 64, 32, 32]             128
              ReLU-4           [-1, 64, 32, 32]               0
            Conv2d-5           [-1, 64, 32, 32]          36,928
       BatchNorm2d-6           [-1, 64, 32, 32]             128
 AdaptiveAvgPool2d-7             [-1, 64, 1, 1]               0
            Conv2d-8              [-1, 4, 1, 1]             260
              ReLU-9              [-1, 4, 1, 1]               0
           Conv2d-10             [-1, 64, 1, 1]             320
          Sigmoid-11             [-1, 64, 1, 1]               0
          CALayer-12           [-1, 64, 32, 32]               0
             RCAB-13           [-1, 64, 32, 32]               0
           Conv2d-1

          Conv2d-818              [-1, 4, 1, 1]             260
            ReLU-819              [-1, 4, 1, 1]               0
          Conv2d-820             [-1, 64, 1, 1]             320
         Sigmoid-821             [-1, 64, 1, 1]               0
         CALayer-822           [-1, 64, 32, 32]               0
            RCAB-823           [-1, 64, 32, 32]               0
          Conv2d-824           [-1, 64, 32, 32]          36,928
     BatchNorm2d-825           [-1, 64, 32, 32]             128
            ReLU-826           [-1, 64, 32, 32]               0
          Conv2d-827           [-1, 64, 32, 32]          36,928
     BatchNorm2d-828           [-1, 64, 32, 32]             128
AdaptiveAvgPool2d-829             [-1, 64, 1, 1]               0
          Conv2d-830              [-1, 4, 1, 1]             260
            ReLU-831              [-1, 4, 1, 1]               0
          Conv2d-832             [-1, 64, 1, 1]             320
         Sigmoid-833             [-1, 6

           RCAB-1537           [-1, 64, 32, 32]               0
         Conv2d-1538           [-1, 64, 32, 32]          36,928
    BatchNorm2d-1539           [-1, 64, 32, 32]             128
           ReLU-1540           [-1, 64, 32, 32]               0
         Conv2d-1541           [-1, 64, 32, 32]          36,928
    BatchNorm2d-1542           [-1, 64, 32, 32]             128
AdaptiveAvgPool2d-1543             [-1, 64, 1, 1]               0
         Conv2d-1544              [-1, 4, 1, 1]             260
           ReLU-1545              [-1, 4, 1, 1]               0
         Conv2d-1546             [-1, 64, 1, 1]             320
        Sigmoid-1547             [-1, 64, 1, 1]               0
        CALayer-1548           [-1, 64, 32, 32]               0
           RCAB-1549           [-1, 64, 32, 32]               0
         Conv2d-1550           [-1, 64, 32, 32]          36,928
    BatchNorm2d-1551           [-1, 64, 32, 32]             128
           ReLU-1552           [-1, 64

        CALayer-2334           [-1, 64, 32, 32]               0
           RCAB-2335           [-1, 64, 32, 32]               0
         Conv2d-2336           [-1, 64, 32, 32]          36,928
    BatchNorm2d-2337           [-1, 64, 32, 32]             128
           ReLU-2338           [-1, 64, 32, 32]               0
         Conv2d-2339           [-1, 64, 32, 32]          36,928
    BatchNorm2d-2340           [-1, 64, 32, 32]             128
AdaptiveAvgPool2d-2341             [-1, 64, 1, 1]               0
         Conv2d-2342              [-1, 4, 1, 1]             260
           ReLU-2343              [-1, 4, 1, 1]               0
         Conv2d-2344             [-1, 64, 1, 1]             320
        Sigmoid-2345             [-1, 64, 1, 1]               0
        CALayer-2346           [-1, 64, 32, 32]               0
           RCAB-2347           [-1, 64, 32, 32]               0
         Conv2d-2348           [-1, 64, 32, 32]          36,928
    BatchNorm2d-2349           [-1, 64

# Gaussian Filter

In [10]:
def matlab_style_gauss2D(shape,sigma=1):
    """ 
    2D gaussian filter - should give the same result as:
    MATLAB's fspecial('gaussian',[shape],[sigma]) 
    """
    m,n = [(ss-1.)/2. for ss in shape]
    y,x = np.ogrid[-m:m+1,-n:n+1]
    h = np.exp( -(x*x + y*y) / (2.*sigma*sigma) )
    h.astype('float32')
    h[ h < np.finfo(h.dtype).eps*h.max() ] = 0
    sumh = h.sum()
    if sumh != 0:
        h /= sumh
    h = h*2.0
    h = h.astype('float32')
    return h

In [11]:
psf_heatmap = torch.from_numpy(matlab_style_gauss2D(shape=(7,7),sigma=1))
gfilter = torch.reshape(psf_heatmap, [1, 1, 7, 7])
zero = torch.zeros(batchSize, 1, 128, 128)
amplitude = 15.0
lmd = 0.1
gfilter = gfilter.cuda()

# Training

In [12]:
def train(epoch):
    epoch_loss = 0
    losses = []
    model.train()
    for iteration, batch in enumerate(training_data_loader, 1):
        input, target = Variable(batch[0].float()).cuda(), Variable(batch[1].float()).cuda()
        zero = torch.zeros(target.size()).cuda()
        
        target *= amplitude
        target = F.conv2d(target, gfilter, padding = 3)
        optimizer.zero_grad()
        out = model(input)
        out2 = F.conv2d(out, gfilter, padding = 3)
        loss = criterion(out2, target) + lmd * criterion2(out, zero)
        epoch_loss += loss.item()
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
#         scheduler.step()
        print("===> Epoch[{}]({}/{}): Loss: {:.4f} ".format(epoch, iteration, len(training_data_loader), loss.item()))
    
    print("===> Epoch {} Complete: Avg. Loss: {:.4f}".format(epoch, epoch_loss / len(training_data_loader)))
    return losses

# Test

In [13]:
def test():
    with torch.no_grad():
        avg_psnr = 0
        epoch_loss_val = 0
        predictions = []
        losses_val = []
        model.eval()
        for batch in testing_data_loader:
            input, target = Variable(batch[0].float()).cuda(), Variable(batch[1].float()).cuda()
            zero = torch.zeros(target.size())
            zero = zero.cuda()
            
            target *= amplitude
            target = F.conv2d(target, gfilter, padding = 3)
            prediction = model(input)
            prediction2 = F.conv2d(prediction, gfilter, padding = 3)
            loss_val = criterion(prediction2, target) + lmd * criterion2(prediction, zero)
            epoch_loss_val += loss_val.item()
            losses_val.append(loss_val.item())
            psnr = 10 * log10(amplitude / loss_val.item())
            avg_psnr += psnr
        print("===> Avg. PSNR: {:.4f} dB : Avg. Loss: {:.4f} ".format(avg_psnr / len(testing_data_loader), epoch_loss_val / len(testing_data_loader)))
        return losses_val


# Checkpoint

In [14]:
# Function to save the model parameters at each epoch
def checkpoint(epoch):
#     model_out_path = "../trained_models/liu/model_07_noise_5_{}.pth".format(epoch)
    model_out_path = "../trained_models/vansloun/model_07_noise_10_{}.pth".format(epoch)
    torch.save(model, model_out_path)
    print("Checkpoint saved to {}".format(model_out_path))

# Run

In [15]:
# Run the training procedure 
if  __name__ == "__main__":
    L = []
    L_val = []
    for epoch in range(1, nEpochs + 1):
        losses = train(epoch)
        L.append(np.mean(losses))
        losses_val = test()
        L_val.append(np.mean(losses_val))
        checkpoint(epoch)
        plt.plot(L)
        plt.plot(L_val)
        plt.show()
        for param_group in optimizer.param_groups:
                print("Current learning rate is: {}".format(param_group['lr']))
        
end = time.time()
print(end - start)

RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 4.00 GiB total capacity; 2.79 GiB already allocated; 17.28 MiB free; 2.79 GiB reserved in total by PyTorch)

# Clean the memory

In [ ]:
del model

In [16]:
import GPUtil
import gc
print(torch.cuda.memory_reserved())
print(torch.cuda.memory_allocated())
GPUtil.showUtilization()
gc.collect()
torch.cuda.empty_cache()
print(torch.cuda.memory_reserved())
print(torch.cuda.memory_allocated())
GPUtil.showUtilization()

2996830208
2993413632
| ID | GPU | MEM |
------------------
|  0 |  5% | 96% |
2996830208
2993413632
| ID | GPU | MEM |
------------------
|  0 |  0% | 96% |
